# HUC Feature Translation Service (FTS) Examples

This is the associated Jupyter Notebook containing seven examples related to querying the HUC Feature Translation Service and using its results to query NASA's Common Metadata Repository (CMR).

In [2]:
%%capture

import sys
!{sys.executable} -m pip install bs4 requests

In [3]:
from bs4 import BeautifulSoup
import requests
import json

## Partial Region Matches in the HUC Feature Translation Service

Below you can input some given region name and query the Feature Translation Service based on it. Below I've used "California" as a partial match, and the output is the list of all regions within the USGS HUC Database that begin with "California".

In [ ]:
###################

# Mimicing a user querying partial matches with California

REGION = "California"

###################

# Query Feature Translation Service and parse JSON response
r = requests.get("https://g6zl7z2x7j.execute-api.us-west-2.amazonaws.com/prod/region/{}".format(REGION))

# Example query from above is:
# https://g6zl7z2x7j.execute-api.us-west-2.amazonaws.com/prod/region/California

# Load response from FTS
response = json.loads(r.text)

# Print all elements in HUC database that partially match California
print(list(response['results'].keys()))

## Search Feature Translation Service for Exact HUC Matches

Here we can define a HUC, or hydrologic unit code, and use this to query the HUC FTS. By defining the parameter _EXACT = True_, we tell the query to no longer search for partial matches like above.

In [ ]:
###################

# Mimicing a user querying exact matches with HUC "180500030105"

HUC = "180500030105"
EXACT = True

###################

# Query Feature Translation Service and parse JSON response
r = requests.get("https://g6zl7z2x7j.execute-api.us-west-2.amazonaws.com/prod/huc/{}?exact={}".format(HUC, EXACT))

# Load response from FTS
response = json.loads(r.text)

# Print all elements in HUC database that exactly match HUC "180500030105"
print(json.dumps(response, indent = 4))

## Search Feature Translation Service for Partial HUC Matches.

In the same way as the previous example, we can query the HUC FTS based on hydrologic unit code. This time however, we can specify the parameter _EXACT = False_ to tell the query to search for partial matches. Thus, by removing the final digit of the previous HUC, we'll achieve several matches (6 to be exact).

In [ ]:
###################

# Mimicing a user querying partial matches with HUC "1805000301"
# This "partial" match is anything that BEGINS with the HUC specified.

HUC = "1805000301"
EXACT = False

###################

# Query Feature Translation Service and parse JSON response
r = requests.get("https://g6zl7z2x7j.execute-api.us-west-2.amazonaws.com/prod/huc/{}?exact={}".format(HUC, EXACT))

# Load response from FTS
response = json.loads(r.text)

# Print all elements in HUC database that partially matches with HUC 1805000301
print(json.dumps(response, indent = 4))

## Search Feature Translation Service for Exact Region Matches

Given that we know an exact region currently in USGS's Watershed Boundary Dataset (WBD), we can use this instead of a HUC to search. In a similar fashion, we can query both by partial and by exact matches. Below is an example of exact matches by region name.

In [ ]:
###################

# Mimicing a user querying exact matches with region "Woods Creek-Skykomish River"

REGION = "Woods Creek-Skykomish River"
EXACT = True

###################

# Query Feature Translation Service and parse JSON response
# Note the change in endpoint from "/prod/huc" to "/prod/region"
r = requests.get("https://g6zl7z2x7j.execute-api.us-west-2.amazonaws.com/prod/region/{}?exact={}".format(REGION, EXACT))

# Load response from FTS
response = json.loads(r.text)

# Print all elements in HUC database that exact matches with region "Woods Creek-Skykomish River"
print(json.dumps(response, indent = 4))

## Search Feature Translation Service for Partial Region Matches

Below is an example of querying the FTS for _partial_ region matches.

In [ ]:
###################

# Mimicing a user querying partial matches with region "California R"
# This "partial" match is anything that BEGINS with the region specified.

REGION = "California R"
EXACT = False

###################

# Query Feature Translation Service and parse JSON response
r = requests.get("https://g6zl7z2x7j.execute-api.us-west-2.amazonaws.com/prod/region/{}?exact={}".format(REGION, EXACT))

# Load response from FTS
response = json.loads(r.text)

# Print all elements in HUC database that partially matches with HUC 1805000301
print(json.dumps(response, indent = 4))

## Query CMR by Polygon

Here is a more useful example of the Feature Translation Service. We can use results obtained from the FTS to then directly and automatically query CMR. Below I'm extracting the polygon representing California Creek-Kuskokwim River (chosen arbitrarily) and using it to search for granules available through the Sentinel-1 mission. 

In [ ]:

###################

COLLECTION_ID = "C1522341104-NSIDC_ECS" # SMAP/Sentinel-1 L2 Radiometer/Radar 30-Second Scene 3 km EASE-Grid Soil Moisture V002
REGION = "California Creek-Kuskokwim River"
EXACT = True

###################

# Query Feature Translation Service and parse JSON response
r = requests.get("https://g6zl7z2x7j.execute-api.us-west-2.amazonaws.com/prod/region/{}?exact={}".format(REGION, EXACT))

# Load response from FTS
response = json.loads(r.text)

# Obtain convex hull polygon from response
polygon = response['results'][REGION]['Convex Hull Polygon']
#polygon = response['results'][REGION]['Visvalingam Polygon']

# Query CMR
# --------- #

#cmr_response = requests.get("https://cmr.earthdata.nasa.gov/search/granules.json?polygon={}&echo_collection_id=C1522341104-NSIDC_ECS&pretty=True".format(polygon))
cmr_response = requests.get("https://cmr.earthdata.nasa.gov/search/granules?polygon={}&echo_collection_id={}&pretty=True".format(polygon, COLLECTION_ID))

# --------- #

# Make it look nice
soup = BeautifulSoup(cmr_response.text, features = 'lxml')
print(soup.prettify())

## Query CMR by Bounding Box

Instead of querying via polygon, we can extract the bounding box of the region and use this to query CMR. Similarly to above, I'm extracting information (this time the bounding box) from the California Creek-Kuskokwim River and am using this to search for granules available through the Sentinel-1 mission. 

I've queried by _region_ in these two examples, however it would be equally valid to query by _HUC_. 

In [4]:
###################

COLLECTION_ID = "C1522341104-NSIDC_ECS" # SMAP/Sentinel-1 L2 Radiometer/Radar 30-Second Scene 3 km EASE-Grid Soil Moisture V002
REGION = "California Creek-Kuskokwim River" 
EXACT = True

###################

# Query Feature Translation Service and parse JSON response
r = requests.get("https://g6zl7z2x7j.execute-api.us-west-2.amazonaws.com/prod/region/{}?exact={}".format(REGION, EXACT))

# Load response from FTS
response = json.loads(r.text)

# Obtain bounding box from response
bbox = response['results'][REGION]['Bounding Box']

# Query CMR
# --------- #

#cmr_response = requests.get("https://cmr.earthdata.nasa.gov/search/granules.json?bounding_box={}&echo_collection_id=C1522341104-NSIDC_ECS&pretty=True".format(bbox))
cmr_response = requests.get("https://cmr.earthdata.nasa.gov/search/granules?bounding_box={}&echo_collection_id={}&pretty=True".format(bbox, COLLECTION_ID))

# --------- #

# Make it look nice
soup = BeautifulSoup(cmr_response.text, features = 'lxml')
print(soup.prettify())

<?xml version="1.0" encoding="UTF-8"?>
<html>
 <body>
  <results>
   <hits>
    36
   </hits>
   <took>
    69
   </took>
   <references>
    <reference>
     <name>
      SC:SPL2SMAP_S.002:137224366
     </name>
     <id>
      G1547570860-NSIDC_ECS
     </id>
     <location>
      https://cmr.earthdata.nasa.gov:443/search/concepts/G1547570860-NSIDC_ECS/6
     </location>
     <revision-id>
      6
     </revision-id>
    </reference>
    <reference>
     <name>
      SC:SPL2SMAP_S.002:138549434
     </name>
     <id>
      G1549595081-NSIDC_ECS
     </id>
     <location>
      https://cmr.earthdata.nasa.gov:443/search/concepts/G1549595081-NSIDC_ECS/6
     </location>
     <revision-id>
      6
     </revision-id>
    </reference>
    <reference>
     <name>
      SC:SPL2SMAP_S.002:137107471
     </name>
     <id>
      G1547458114-NSIDC_ECS
     </id>
     <location>
      https://cmr.earthdata.nasa.gov:443/search/concepts/G1547458114-NSIDC_ECS/6
     </location>
     <revision-id>
 